# _Hi Joyce!!! :^)_

So just to re-hash the thinking of this project:
___

1) Numerical programs are hard to debug, because they can have purely *numerical* errors, i.e. the code runs fine, but spits out answers that are numerically wrong.

2) We can alleviate this by introducing "tracebacks" for calculations, that show the intermediate values in a calculation.

3) To do this, I'm introducing a wrapper type called "Calc" that has the same interface as a numerical or array value, but keeps track of its own history.
___

## Prototype:

Here's the code for my prototype, which you can ignore for now bc it's ugly and the demo below should explain it better. For the prototype, I only implemented addition (and half of "less-than") using operator overloading. A fully realized version would overload all the same operators that work for ints/floats.

The "correct" way to create a Calc object in this protoype is to use the c() function, but the user interface could definitely change and I'm open to comments on it!

Anyway. Feel free to look over the code or just skip to the demo below.

In [118]:
import ast

class Calc:
    def __init__(self, expr, value, inputs):
        self.expr = expr
        self.value = value
        self.inputs = inputs
        self.alias = None

    def __add__(self, other):
        return Calc(f'({self.alias if self.alias is not None else self.value}) + {str(other)}',
                    self.value + (other.value if isinstance(other, Calc) else other),
                    [self] + ([other] if isinstance(other, Calc) else []))

    def __radd__(self, other):
        return Calc(f'{str(other)} + ({self.alias if self.alias is not None else self.expr})',
                    self.value + (other.value if isinstance(other, Calc) else other),
                    [self] + ([other] if isinstance(other, Calc) else []))
    
    def __lt__(self, other):
        return Calc(f'({self.alias if self.alias is not None else self.expr}) < {str(other)}',
                    self.value < (other.value if isinstance(other, Calc) else other),
                    [self] + ([other] if isinstance(other, Calc) else []))

    def __repr__(self):
        result = f'Calc("{self.expr}"'
        if self.alias is not None:
            result += f', "{self.alias}"'
        result += ')'
        return result

    def __str__(self):
        return str(self.value)

    def tb(self):
        result = f"{self.alias + ' = ' if self.alias is not None else ''}{self.expr} = "
        len_header = len(result)

        val = str(self.value).split('\n')
        result += val[0]
        for line in str(self.value).split('\n')[1:]:
            result += '\n' + (' ' * len_header) + line

        # recurse to inputs
        for i in self.inputs:
            result += "\n"
            for line in i.tb().split('\n'):
                result += f'\t{line}\n'
        return result

def c(expr, alias=None):
    st = ast.parse(expr)
    all_vars = dict(locals(), **globals())
    inputs = {all_vars[node.id]
              for node in ast.walk(st)
              if type(node) is ast.Name
                  and node.id in all_vars
                  and type(all_vars[node.id]) is Calc}

    value = eval(expr)
    result = Calc(expr, value.value if isinstance(value, Calc) else value, inputs)
    result.alias = alias
    return result

# Demo (Basic Interface)

The main attraction is the `Calc` object, which wraps around a numeric value. These are instantiated using the `c(expr)` function. Also, `Calc` objects can optionally be given "aliases" (variable names).

In [108]:
c("1")

Calc("1")

In [109]:
c("1", "A")

Calc("1", "A")

`Calc` objects can be used in calculations just like normal numbers. The result is a Calc object, and the value can be retrieved using `.value`.

In [110]:
a = c("1") + 3.2
print(a.__repr__())
print(a.value)

Calc("(1) + 3.2")
4.2


This also works for numpy arrays. (In fact, any expression with a numeric or array value can be used to create a `Calc` object.)

In [111]:
import numpy as np

E = c("np.zeros([2, 2])", "E")
print((E + 1).__repr__())
print((E + 1).value)

Calc("(E) + 1")
[[ 1.  1.]
 [ 1.  1.]]


The key advantage to using Calc objects is that unlike normal numeric variables, they remember the history of calculations made to produce their value. This can be retrieved using the traceback function `tb()`.

In [112]:
A = c("1 + 2", "A")
B = c("A + 3", "B")
C = c("B + 4", "C")
print(C.tb())

C = B + 4 = 10
	B = A + 3 = 6
		A = 1 + 2 = 3
	



~As you can see~, `C` remembers that it was made by adding 4 to `B`, which was in turn made by adding 3 to `A`, which was made by calculating `1 + 2`. Along the right-hand-side of the printout, the intermediate values (`A=3`, `B=6`, `C=10`) can be seen.

Using the `alias` option makes the `tb()` printout more readable, but it works either way:

In [113]:
G = c("1")
H = G + 123
I = H + 44
print(I.tb())

(124) + 44 = 168
	(1) + 123 = 124
		1 = 1
	



This second case may be easier for the user in a debugging context, since they can just `Calc`-ify the constants going into their calculation, and the result automatically becomes a `Calc` that they can do tracebacks on. I imagine that the first case, of providing names for all the intermediate steps, as a fallback if the information from this lazy way isn't informative enough.

# Practical Demo

Okay so Bobby is in CS106A, and they're learning recursion by doing the Fibonacci sequence. But oh no, there's a typo!


Bobby's function to find the nth Fibonacci number does $F_n = F_{n-1} + F_{n-3}$ instead of $F_n = F_{n-1} + F_{n-2}$.

In [122]:
def BobbyFibb(n):
    if n <= 1:
        return 1
    return BobbyFibb(n - 1) + BobbyFibb(n - 3) # whoops
    
BobbyFibb(4)

4

Bobby checks a reference and finds that actually, the 4th Fibonacci number is 5 and not 4. Bobby racks their brains for hours trying to find the error!

Eventually, they remmeber this cool new library for debugging that they heard about, and they decide to try it out. Bobby uses the `c(expr)` function to turn the base-case constant 1 into a Calc object.

In [137]:
def BobbyFibb(n):
    if n <= 1:
        return c("1", "Base")
    return BobbyFibb(n - 1) + BobbyFibb(n - 3) # whoops

Now, Bobby can do a traceback on the output, to see what intermediate calculations were made:

In [138]:
print(BobbyFibb(4).tb())

(3) + 1 = 4
	(2) + 1 = 3
		(Base) + 1 = 2
			Base = 1 = 1
		
			Base = 1 = 1
		
	
		Base = 1 = 1
	

	Base = 1 = 1



4 was made by adding 1 to 3. Immediately, this looks fishy because 3 does not follow 1 in the Fibonacci sequence. Let's go back further: 3, in turn, was made by adding 1 to 2, which was made by doing 1 + 1. So, the overall calculation was 1 + 1 + 1 + 1 = 4. This suggests to Bobby that we're hitting the base case far more often than we should be, which hopefully helps them to realize that the recursive part reaches further back than it should.

Once Bobby finds and corrects the error, then Bobby is free to undo the change making the base case into a `Calc`, and voila! A working Fibonacci number function.

In [136]:
def BobbyFibb(n):
    if n <= 1:
        return 1
    return BobbyFibb(n - 1) + BobbyFibb(n - 2)

print(BobbyFibb(4))
print("yay!")

5
yay!
